In [2]:
from pathlib import Path
import potentials
import atomman as am

__\_\_schema\_\___ contains the XSD schemas associated with the XML 
representations of the different record types.  These schemas can be used to
check and verify if records are comliant with the format.

In [3]:
machine = 'laptop'

# Set passwordfile based on machine
if machine == 'laptop':
    passwordfile = Path('C:/Users/lmh1/Documents/potentials_nist_gov/password.txt')
elif machine == 'desktop':
    passwordfile = Path('E:/potentials_nist_gov/password.txt')
else:
    raise ValueError(f'passwordfile not set for machine {machine}')

# Load database using username + password
with open(Path(passwordfile)) as f:
    username, password = f.read().strip().split()
potdb = potentials.Database(username=username, password=password, status=None, installed_kim_models=[])

In [3]:
potdb.lammps_potentials

In [4]:
potdb.load_lammps_potentials(remote=True, local=False, verbose=True)

Loaded 401 remote LAMMPS potentials
Building lammps potentials for kim models
No installed kim models identified


In [6]:
workspace = potdb.cdcs.global_workspace

In [8]:
potdb.cdcs.assign_records(template='potential_LAMMPS_KIM', workspace=workspace, verbose=True)

record 605b6bf326ed1e002a28f053 assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b6bef26ed1e003528f019 assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b6bec26ed1e003d28f018 assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b6be826ed1e003928f033 assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b6be426ed1e003c28f015 assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b6be226ed1e002a28f04f assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b6bde26ed1e003528f015 assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b6bdc26ed1e003d28f014 assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b6bd926ed1e003928f02f assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b6bd426ed1e003c28f011 assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b6bd126ed1e002a28f04b assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b6bcc26ed1e003528f011 assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b6bc826ed1e003d28f010 assigned to workspac

record 605b690f26ed1e003528ef75 assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b690b26ed1e003d28ef74 assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b690926ed1e003928ef8f assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b690626ed1e003c28ef71 assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b690326ed1e002a28efab assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b68ff26ed1e003528ef71 assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b68fc26ed1e003d28ef70 assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b68f926ed1e003928ef8b assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b68f526ed1e003c28ef6d assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b68f126ed1e002a28efa7 assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b68ed26ed1e003528ef6d assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b68ea26ed1e003d28ef6c assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b68e626ed1e003928ef87 assigned to workspac

record 605b662326ed1e003928ef0b assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b662026ed1e003828ef15 assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b661d26ed1e002a28ef1f assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b661a26ed1e003a28ef06 assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b661726ed1e003b28ef0c assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b661326ed1e003928ef07 assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b661026ed1e003828ef11 assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b660d26ed1e002a28ef1b assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b660a26ed1e003a28ef02 assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b660726ed1e003b28ef08 assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b660326ed1e003928ef03 assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b660026ed1e003828ef0d assigned to workspace 5fb55e4826ed1e0015e846a9
record 605b65fd26ed1e002a28ef17 assigned to workspac

In [5]:
print(potdb.lammps_potentials[0].pair_info())

print "Potential 1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1 listed in the NIST Interatomic Potentials Repository:"
print "https://www.ctcms.nist.gov/potentials/entry/1985--Foiles-S-M--Ni-Cu/1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1.html"
print "Publication(s) related to the potential:"
print "https://doi.org/10.1103/physrevb.32.7685"
print "Parameter file(s) can be downloaded at:"
print "https://www.ctcms.nist.gov/potentials/Download/1985--Foiles-S-M--Ni-Cu/1/Cu_smf7.eam"
print "https://www.ctcms.nist.gov/potentials/Download/1985--Foiles-S-M--Ni-Cu/1/Ni_smf7.eam"
pair_style eam
pair_coeff 1 1 C:\Users\lmh1\Documents\library\potential_LAMMPS\1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1\Cu_smf7.eam
pair_coeff 2 2 C:\Users\lmh1\Documents\library\potential_LAMMPS\1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1\Ni_smf7.eam
mass 1 63.55
mass 2 58.71




In [7]:
d = {}
potdb.widget_lammps_potential(d)

Output()

Dropdown(description='Element1:', options=('', 'Ac', 'Ag', 'Al', 'Am', 'Ar', 'As', 'At', 'Au', 'B', 'Ba', 'Be'…

Dropdown(description='Element2:', options=('', 'Ac', 'Ag', 'Al', 'Am', 'Ar', 'As', 'At', 'Au', 'B', 'Ba', 'Be'…

Dropdown(description='Element3:', options=('', 'Ac', 'Ag', 'Al', 'Am', 'Ar', 'As', 'At', 'Au', 'B', 'Ba', 'Be'…

Dropdown(description='Pair Style:', options=('', 'adp', 'aenet', 'agni', 'bop', 'comb3', 'eam', 'eam/alloy', '…

Dropdown(description='Potential:', options=('1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1', '1985--Stillinger-F-H--Si…

Button(description='Copy Files', style=ButtonStyle())

Output()

In [10]:
print(d['lammps_potential'].pair_info())

print "Potential 2010--Olsson-P-A-T--Au--LAMMPS--ipr1 listed in the NIST Interatomic Potentials Repository:"
print "https://www.ctcms.nist.gov/potentials/entry/2010--Olsson-P-A-T--Au/2010--Olsson-P-A-T--Au--LAMMPS--ipr1.html"
print "Publication(s) related to the potential:"
print "https://doi.org/10.1063/1.3460127"
print "Parameter file(s) can be downloaded at:"
print "https://www.ctcms.nist.gov/potentials/Download/2010--Olsson-P-A-T--Au/2/Au_Olsson_JAP2010.eam.alloy"
pair_style eam/alloy
pair_coeff * * C:\Users\lmh1\Documents\library\potential_LAMMPS\2010--Olsson-P-A-T--Au--LAMMPS--ipr1\Au_Olsson_JAP2010.eam.alloy Au
mass 1 196.96655




In [17]:
pot = potentials.build_lammps_potential('eam/alloy', elements='Al', #comments='Test build potential', #dois='pasf/asfd3',
                                        paramfile='Al.eam.alloy')

In [18]:
pot = pot.potential()

In [19]:
print(pot.pair_info())

pair_style eam/alloy
pair_coeff * * Al.eam.alloy Al
mass 1 26.9815385




In [ ]:
print(potdb.lammps_potentials[400].pair_data_info('test.dat', [True,False, True]))

In [ ]:
print(potdb.lammps_potentials[4].pair_data_info('test.dat', [True,False, True]))

In [ ]:
print(potdb.lammps_potentials[587].pair_data_info('f.dat', [True, True, True], 'Sg'))

In [ ]:
potdb.load_potentials(remote=False)

In [ ]:
lmppot = potdb.lammps_potentials[0]

In [ ]:
pot = potdb.potentials[potdb.potentials_df.id == lmppot.potid][0]

In [ ]:
f'https://doi.org/{pot.citations[0].doi}'

In [1]:
from DataModelDict import DataModelDict as DM
from pathlib import Path

In [2]:
for fname in Path('C:/Users/lmh1/Documents/library/potential_LAMMPS').glob('*.json'):
    with open(fname) as f:
        content = f.read()
    content = content.replace('"comment":', '"comments":')
    
    with open(fname, 'w') as f:
        f.write(content)

In [ ]:
for lmppot in potdb.lammps_potentials:
    new = update_record(lmppot, potdb)
    
    fname = Path(potdb.localpath, 'potential_LAMMPS', f'{new["id"]}.json')
                 
    with open(fname, 'w') as f:
        model = DM([('potential-LAMMPS', new)])
        model.json(fp=f, indent=4)

In [ ]:
potdb.lammps_potentials[0].id

def update_record(lmppot, potdb):
    new = DM()
    
    # Insert comment after potential
    for key in lmppot.model:
        new[key] = lmppot.model[key]
        
        if key == 'potential':
            comment = f'Potential {lmppot.id} listed in the NIST Interatomic Potentials Repository:\n'
            comment += f'https://www.ctcms.nist.gov/potentials/entry/{lmppot.potid}/{lmppot.id}.html\n'
            new['comment'] = comment
    
    # Add doi values to potential
    try:
        pot = potdb.potentials[potdb.potentials_df.id == lmppot.potid][0]
    except:
        pass
    else:
        if 'doi' in new['potential']:
            del new['potential']['doi']
        for citation in pot.citations:
            try:
                new['potential'].append('doi', citation.doi)
            except:
                pass
        
    return new

new = update_record(potdb.lammps_potentials[0], potdb)

In [ ]:
new

In [ ]:
for i in potdb.lammps_potentials_df[potdb.lammps_potentials_df.pair_style == 'kim'].index[180:200]:
    print(i, potdb.lammps_potentials_df.loc[i, 'symbols'])

In [ ]:
potdb.lammps_potentials_df[potdb.lammps_potentials_df.id == 'EAM_Dynamo_MendelevAckland_2007v3_Zr__MO_004835508849_000'].index

In [ ]:
from pathlib import Path
from DataModelDict import DataModelDict as DM
import potentials

In [ ]:
potdb = potentials.Database(load=['potentials', 'lammps_potentials'], remote=False)

In [ ]:
fnames = [i for i in Path('C:/Users/lmh1/Documents/library/potential_LAMMPS_KIM').glob('*.json')]

In [ ]:
for fname in Path('C:/Users/lmh1/Documents/library/potential_LAMMPS_KIM').glob('*.json'):
    
    model = DM(fname)
    
    stop = False
    for kimpot in model['potential-LAMMPS-KIM'].aslist('potential'):
        if 'atom' in kimpot:
            continue
        if 'elements' not in kimpot:
            print('No elements listed')
            stop = True
            break
            
        else:
            print(fname.stem)
            # Find the corresponding potentials listing
            try:
                pot = potdb.potentials[potdb.potentials_df.key == kimpot['key']][0]
            except:
                print('No matching potential listing')
                stop = True
                break
            
            
            impids = []
            for imp in pot.implementations:
                impids.append(imp.id)
            try:
                lmppot = potdb.lammps_potentials[potdb.lammps_potentials_df.id.isin(impids)][0]
            except:
                for element in kimpot.aslist('elements'):
                    kimpot.append('atom', DM([('element', element)]))
                del kimpot['elements']
            else:
                del kimpot['elements']
                kimpot['atom'] = lmppot.model['atom']
    if stop:
        break
    with open(fname, 'w') as f:
        model.json(fp=f, indent=4)

In [ ]:
for element in kimpot.aslist('elements'):
    kimpot.append('atom', DM([('element', element)]))
del kimpot['elements']

In [ ]:
with open(fname, 'w') as f:
    model.json(fp=f, indent=4)

In [ ]:
for potential in model['potential-LAMMPS-KIM'].aslist('potential'):
    pot = potdb.potentials[potdb.potentials_df.key == potential['key']][0]
    impids = []
    for imp in pot.implementations:
        impids.append(imp.id)
    lmppots = potdb.lammps_potentials[potdb.lammps_potentials_df.id.isin(impids)][0]
lmppots.model['atom']

In [ ]:
lammps_potential = potdb.get_lammps_potential(id='1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1', pot_dir_style='working')

In [ ]:
potdb.get_lammps_potential_files(lammps_potential, verbose=True, local=False)

In [ ]:
isinstance(0, bool)

In [ ]:
s = time.time()
potdb.get_lammps_potentials(forceremote=True)
e = time.time()
print(e-s, 'seconds to download records for all LAMMPS potentials')

In [ ]:
potdb.lammps_potentials

In [ ]:
s = time.time()
potdb.save_lammps_potentials(potdb.lammps_potentials, downloadfiles=True, localpath='test', format='json', indent=4)
e = time.time()
print(e-s, 'seconds to download parameter files for all LAMMPS potentials')

In [ ]:
s = time.time()
potdb.save_lammps_potentials(potdb.lammps_potentials, downloadfiles=True, localpath='test', format='json', indent=4)
e = time.time()
print(e-s, 'seconds to download parameter files for all LAMMPS potentials')

In [ ]:
7720/60/60

In [ ]:
Path('')

In [ ]:
lammps_potential.pot_dir = lammps_potential.id

In [ ]:
lammps_potential.download_files()

In [ ]:
potdb.get_lammps_potential_files(lammps_potential, local=False, verbose=True)

In [ ]:
print(lammps_potential.pair_info())

In [ ]:
import potentials

In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
def load_kim_lammps_potentials(self, localpath=None, local=None, remote=None, verbose=False):

    installed_kim_models = self.installed_kim_models
    
    # Return empty fields if no installed kim models found
    if len(installed_kim_models) == 0:
        return [], pd.DataFrame()

    # Get potential_LAMMPS_KIM records
    records = db.get_records(template='potential_LAMMPS_KIM', localpath=localpath, local=local, 
                             remote=remote)
    
    # Convert to dict
    recorddict = {}
    for record in remote_records:
        recorddict[record['potential-LAMMPS-KIM']['id']] = record
        
    lammps_potentials = []
    lammps_potentials_df = []

    for fullid in installed_kim_models:
        if '__MO_' in fullid:
            shortcode = '_'.join(fullid.split('_')[-3:-1])

            if shortcode in recorddict:
                try:
                    lammps_potential = potentials.PotentialLAMMPSKIM(recorddict[shortcode], fullid)
                    lammps_potentials.append(lammps_potential)
                    lammps_potentials_df.append(lammps_potential.asdict())
                except:
                    if verbose:
                        print(fullid, 'failed to build')
            else:
                if verbose:
                    print(fullid, 'unknown')
    
    if verbose:
        print(f'Loaded {len(lammps_potentials)} installed KIM LAMMPS potentials')
        
    return lammps_potentials, lammps_potentials_df

In [ ]:
load_kim_lammps_potentials(verbose=True)

In [ ]:
machine = 'laptop'

In [ ]:
# Set passwordfile based on machine
if machine == 'laptop':
    passwordfile = Path('C:/Users/lmh1/Documents/potentials_nist_gov/password.txt')
elif machine == 'desktop':
    passwordfile = Path('E:/potentials_nist_gov/password.txt')
else:
    raise ValueError(f'passwordfile not set for machine {machine}')

# Load database using username + password
with open(Path(passwordfile)) as f:
    username, password = f.read().strip().split()
db = potentials.Database(username=username, password=password)

In [ ]:
remote_records = db.get_records(template='potential_LAMMPS_KIM', verbose=True)

In [ ]:
# Convert to dict
recorddict = {}
for record in remote_records:
    recorddict[record['potential-LAMMPS-KIM']['id']] = record

In [ ]:
lammps_potentials = []
lammps_potentials_df = []

for fullid in installed_kim_models:
    if '__MO_' in fullid:
        shortcode = '_'.join(fullid.split('_')[-3:-1])
        
        if shortcode in recorddict:
            try:
                lammps_potential = potentials.PotentialLAMMPSKIM(recorddict[shortcode], fullid)
                lammps_potentials.append(lammps_potential)
                lammps_potentials_df.append(lammps_potential.asdict())
            except:
                print(fullid, 'failed to build')
        else:
            print(fullid, 'unknown')

In [ ]:
lammps_potentials_df = pd.DataFrame(lammps_potentials_df)
lammps_potentials_df

In [ ]:
import atomman as am

In [ ]:
am.tools.atomic_mass('Am')